# 1- Model settings

We setup a source-to-sink case. The initial surface is 300 km by 200 km with the space resolution of 0.5 km. It includes a mountain range (source area), a gently-sloping coastal plain (transfer zone), and a continental margin consisting of a continetal shelf, a slope and an oceanic basin (sink area).
  
<img src="images/Geometry.png" alt="Initial surface"  width="600" height="500" border="10" />

See specific parameters in the table below:

|   | Mountain | Plain | Continental shelf | Continental slope | Oceanic basin|
| :-----------: |:-----------:|:-----------:|:-----------:|:-----------:|:-----------:|
|Width/km| 100 | 50 | 100 | 30 | 20 |
|Elevation/m| 1000~200 | 200~0 | 0~-250 | -250~-1000 | -1000|
|Slope| 0.008 | 0.004 | 0.0025 | 0.025 | 0.0 |

The model duration spans over 30 Myr with display intervals every 0.1 Myr. Forcing conditions include:
* climate: spatially and temporally uniform rainfall rate of **2.0 m/yr**;
* sea-level input, see **1.1**;
* thermal subsidence over continental margin, see **1.2**.

## 1.1- Generate a sea-level curve

Here we consider the effects of long-term eustatic cycles and assume that eustasy is independent of climate and local tectonics. For simplicity, the eustatic sea level is modeled using a sinusoidal curve. It consists of three 10 Myr cycles of 50 m amplitude, which correspond to second- to third-order eustatic fluctuations ([Vail, etal., 1977](http://archives.datapages.com/data/specpubs/seismic1/data/a165/a165/0001/0050/0083.htm)).

The sea-level file (**CSV format**) provides for each line the following information:
* Time in years,
* Sea-level elevation for the considered time in meters.

Detailed information of creating a sea-level curve can be found in [**this**](https://github.com/badlands-model/pyBadlands-Companion/blob/master/notebooks/seaLevel.ipynb) notebook.

In [ ]:
import numpy as np
import matplotlib.mlab as ml
import matplotlib.pyplot as plt
%matplotlib inline

# Import badlands grid generation toolbox
import badlands_companion.toolSea as tools

# display plots in SVG format
%config InlineBackend.figure_format = 'svg' 

In [ ]:
sea = tools.toolSea()

# Create the sea-level curve 
sea.buildCurve(timeExt = [0.,30000000.], timeStep = 100000., seaExt = [-25.,-25.], 
                   ampExt = [25.,25.], periodExt = [10000000.,10000000.])

# Visualize the sea-level curve
sea.plotCurves(fsize=(6,8), figName = 'Sea level')

In [ ]:
# Export sea-level dataset
sea.exportCurve(nameCSV='sealevel')

## 1.2- Generate thermal subsidence curve

A simple stretching model from McKenzie ([1978](https://www.sciencedirect.com/science/article/pii/0012821X78900717)) is applied in this study, in which subsidence is produced by thermal relaxation following an episode of extension. The equation to derive the subsidence $S$ at time $t$ is: 

$$ S(t) = E_0 r - E_0 e^{(-t/\tau)} $$ 

where $E_0 = 4a\rho_0\alpha T_1/\pi^2(\rho_0-\rho_w)$, $r = \frac{\beta}{\pi}sin(\frac{\pi}{\beta})$, $a = 125$km is the thickness of lithosphere, $\rho_0 = 3300$ $\mathrm{kg/m^{3}}$ the density of the mantle at 0 $\mathrm{^oC}$, $\rho_w =1000$ $\mathrm{kg/m^{3}}$ the density of seawater, $\alpha = 3.28\times10^{-5}$ $\mathrm{K^{-1}}$ the thermal expansion coefficient for both the mantle and the crust, $T_1 = 1333$ $\mathrm{^oC}$ the temperature of the asthenosphere, and $\tau = 62.8$ Myr the characteristic thermal diffusion time.



In [ ]:
# Calculate thermal subsidence over continental margin
E0 = 3200.0  # m
tao = 62.8e6  # yr
dist = np.linspace(0,150,301) 
beta = 1.0/100.0*dist+1.0
r = (beta/np.pi)*np.sin(np.pi/beta)
subsidence = (E0*r*np.exp(0/tao) - E0*r*np.exp(-30e6/tao))*2/3

# Visualize total thermal subsidence in 30 Myr along the margin
plt.plot(dist,subsidence)
plt.xlim(0, 150)
plt.ylim(650, 0)
plt.xlabel("Distance from initial shoreline (km)")
plt.ylabel("Subsidence (m)")

In [ ]:
# Export thermal subsidence dataset
f = open("thermsub"+".csv", "w")
for j in range(0,401):
    for i in range(0,300):
        f.write( str(0) + "\n"  )
    for i in range(300,601):
        f.write( str(-subsidence[i-300]) + "\n"  )
f.close()

# 2- Start pyBadlands

In [ ]:
from badlands.model import Model as badlandsModel

# Initialise model
model = badlandsModel()
# Define the XmL input file
model.load_xml('input_gmd.xml')

# 3- Run the model

In [ ]:
# Run model for 30 Myr.
model.run_to_time(30000000)